In [1]:
import numpy as np
import os
from lib.utils import process_sensor_data
df_dict = {}
# Define path to parent directory containing subdirectories with CSV files
parent_dir = 'sc_sensor'
# adding to the df_dict
# Loop through each subdirectory in the parent directory


In [2]:
from lib.utils import gen_data_dict, seperate_up_down
import pickle
# dataset_name = "train_station"
dataset_name = "maze"
if dataset_name != "maze":
    df_dict = process_sensor_data(parent_dir, df_dict)  # only process sensor 2
    data_dict = gen_data_dict(df_dict)
    # process data dict to get the upstream and downstream data
    from lib.utils import seperate_up_down, generating_ood_dataset
    data_dict = seperate_up_down(data_dict)
elif dataset_name == "maze":
    with open("./sc_sensor/maze/flow_data.pkl", "rb") as f:
        data_dict = pickle.load(f)

In [3]:
from torch.utils.data import DataLoader
from lib.dataloader import FlowDataset
from lib.utils import gen_data_dict, process_sensor_data, StandardScaler, sliding_win
from lib.utils import generating_ood_dataset, seperate_up_down, generating_insample_dataset, get_trainable_params_size
import dgl
import torch
from dgl.data.utils import load_graphs
# import pickle
# set seed
torch.manual_seed(1)
#normalization
# x_scalar = StandardScaler(mean=np.concatenate([x_train, x_val]).mean(),
#                           std=np.concatenate([x_train, x_val]).std())

# dataset_name = "crossroad"

g_data = load_graphs('./graphs/graphs.bin')    
if dataset_name == "crossroad":
    # file_path = 'graphs/graph_data_crossroad.pkl'
    # with open(file_path, 'rb') as file:
    #     g = pickle.load(file)
    g = g_data[0][0]

elif dataset_name == "train_station":
    # file_path = 'graphs/graph_data_trainstation.pkl'
    # with open(file_path, 'rb') as file:
    #     g = pickle.load(file)
    g = g_data[0][1]

elif dataset_name == "maze":
    # file_path = 'graphs/graph_data_maze.pkl'
    # with open(file_path, 'rb') as file:
    #     g = pickle.load(file)
    g = g_data[0][2]


In [4]:
if dataset_name == "crossroad":
    train_sc = ['sc_sensor/crossroad2']
    test_sc = ['sc_sensor/crossroad2_2']

if dataset_name == "train_station":
    train_sc = ['sc_sensor/train6']
    test_sc = ['sc_sensor/train3']
    
if dataset_name == "maze":
    train_sc = ['sc_sensor/maze0']
    test_sc = ['sc_sensor/maze0', 'sc_sensor/maze5','sc_sensor/maze8', 'sc_sensor/maze9',
               'sc_sensor/maze10', ]

# x_train, y_train, x_val, y_val, x_test, y_test = generating_ood_dataset(data_dict, train_sc, test_sc, lags=5, horizons=pred_horizon, shuffle=True)
# x_train, y_train, x_val, y_val, x_test, y_test = generating_insample_dataset(data_dict, train_sc,
#                                                                              lags=5,
#                                                                              horizons=pred_horizon,
#                                                                              portion=0.6,
#                                                                              shuffle=True)

chunk_size = 30
lags = 5
# '''Has to >= 2'''
pred_horizon = 7 # 3, 5
x_test, y_test = sliding_win(data_dict["sc_sensor/maze16"], lags=lags, horizons=pred_horizon)
num_input_timesteps = x_test.shape[1] # number of input time steps
num_nodes = x_test.shape[2] # number of ancestor nodes, minus the down stream node
# x_scalar = StandardScaler(mean=np.concatenate([x_train, x_val]).mean(),
#                           std=np.concatenate([x_train, x_val]).std())


In [5]:
# from Diffusion_Network import Diffusion_Model
# from Diffusion_Network2 import Diffusion_Model
# from Diffusion_Network3_asc import Diffusion_Model
from Diffusion_Network4 import Diffusion_Model
# from Diffusion_Network_density import Diffusion_Model_Density
# from Diffusion_Network_UQ import Diffusion_Model_UQ
src, dst = g.edges()
# model = Diffusion_Model_UQ(num_edges=len(src), num_timesteps_input=num_input_timesteps, graph=g, horizons=pred_horizon, scalar=None)

# model = Diffusion_Model_Density(num_edges=len(src), num_timesteps_input=num_input_timesteps, graph=g, horizons=pred_horizon, scalar=None)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Diffusion_Model(num_edges=len(src), num_timesteps_input=x_test.shape[1], graph=g, horizons=pred_horizon, device=device, scalar=None)
# chunk_size = 30
# lags = 5
# '''Has to >= 2'''
# pred_horizon = 5 # 3, 5
# model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_model_network3.pth"))
if dataset_name == "crossroad":
    # model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_model_network3_cross.pth"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/online_diffusion.pt"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/offline_diffusion_uq_cross.pth"))
    model.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
if dataset_name == "train_station":
    # model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_model_network4.pth"))
    model.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/online_diffusion_density.pth"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_model_density.pth"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_uq.pth"))
if dataset_name == "maze":
    model.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
    # model.load_state_dict(torch.load("./checkpoint/diffusion/diffusion_model_network4_maze.pth"))

In [13]:
with torch.no_grad():
    x_up = torch.FloatTensor(x_test[..., src].transpose(2, 0, 1)) # [num of src, batch_size, num_timesteps_input], num of src = num of dst = num of edges
    x_down = torch.FloatTensor(x_test[..., dst].transpose(2, 0, 1)) # [num of dst, batch_size, num_timesteps_input]
    x = torch.FloatTensor(x_test.transpose(2, 0, 1))
    y = torch.FloatTensor(y_test.transpose(2, 0, 1))
    g.ndata['feature'] = x # [node, batch_size, num_timesteps_input]
    g.ndata['label'] = y # [node, batch_size, pred_horizon]
    # v = model.velocity_model(x_up, x_down).numpy()
    model(g.ndata['feature'][src], g.ndata['feature'][dst])
    # flow = torch.sum(x_up, dim=2).numpy()
    total_flow = torch.mean(torch.cat([x_up, x_down], dim=-1), dim=-1).numpy()
flow = model.g.edata['message']
speed = model.g.edata['v']
F = model.g.edata['F']

In [7]:
import matplotlib.pyplot as plt
src_point =22
dst_point = 2
edge_id = np.where((src == src_point) & (dst == dst_point))[0]
# plt.plot(flow[edge_id, :250].squeeze(), speed[edge_id, :250].squeeze(), '*')
# plt.plot(flow[edge_id, 250:].squeeze(), speed[edge_id, 250:].squeeze(), '>')
plt.xlabel('Flow')
plt.ylabel('Speed')
plt.plot(flow[edge_id, :].squeeze(), speed[edge_id, :].squeeze(), '>')

In [8]:
#netowrk2
# g.ndata['feature'] = torch.FloatTensor(x_test.transpose(2, 0, 1))
# pred = model.inference(g.ndata['feature'][src], g.ndata['feature'][dst])

In [9]:
#network
# g.ndata['feature'] = torch.FloatTensor(x_test.transpose(2, 0, 1))
# g.ndata['label'] = torch.FloatTensor(y_test.transpose(2, 0, 1))
# z = model.transition_probability(g.ndata['feature'])
# g.ndata['embedding'] = z   # for attention
# g.apply_edges(model.transition_probability.edge_attention)

In [11]:
# generate matrix with the element is e
with torch.no_grad():
    num_edges = len(src)
    sample_size = x_test.shape[0]
    atten_mat = torch.zeros((num_nodes, num_nodes, sample_size))
    for i in range(num_edges):
        atten_mat[src[i], dst[i], :] = g.edata['e'][i, :]
    # atten_mat = atten_mat.numpy()

In [83]:
# generate matrix with the element is e
np.around(atten_mat[..., -1].numpy(), 3)

In [8]:
with torch.no_grad():
    num_edges = len(src)
    sample_size = x_test.shape[0]
    alpha_mat = torch.zeros((num_nodes, num_nodes, sample_size))
    for i in range(num_edges):
        # alpha_mat[src[i], dst[i], :] = g.edata['alpha'][i, :]
        alpha_mat[src[i], dst[i], :] = g.edata['F'][i, :]


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 8))
# sns.heatmap(alpha_mat[..., -1], cmap="YlGnBu")
sns.heatmap(torch.mean(alpha_mat, dim=-1), cmap="YlGnBu")

In [12]:
# plt.figure(figsize=(10, 8))
sns.heatmap(atten_mat[..., -1], cmap="YlGnBu")

In [15]:
sns.heatmap(np.multiply(atten_mat[..., -1].numpy(), alpha_mat[..., -1].numpy()), cmap="YlGnBu")

In [46]:
import dgl
import torch

# Create a simple DGLGraph
# num_nodes = 5
# edges_src = [0, 1, 1, 2, 2, 3, 3, 4]
# edges_dst = [1, 2, 3, 3, 4, 4, 0, 1]
# graph = dgl.graph((edges_src, edges_dst), num_nodes=num_nodes)
# graph.edata['features'] = torch.randn(8, 10)
# src_idx, dst_idx = g.edges()
src_idx = np.unique(src)
# Get edge IDs going out from a specific source node
edge_ids_dict = dict()
for src_node_id in src_idx:
    outbound_edges = np.where(np.array(src) == src_node_id)[0]
    edge_ids_dict[src_node_id] = outbound_edges
    print(src_node_id)
    edge_data = g.edata['e'][outbound_edges, :]
    print(torch.sum(edge_data))
    # assert torch.sum(edge_data) != 354
    print("Edge Data:", edge_data.shape)
    print("outbound_edges:", outbound_edges)

# Extract edge features and destination nodes connected to the specified source node
# edge_data = graph.edata['features'][outbound_edges]
# src_nodes = graph.edges()[0][outbound_edges]
# dst_nodes = graph.edges()[1][outbound_edges]

# print("Edge Data:", edge_data)
# print("Source Nodes:", src_nodes)
# print("Destination Nodes:", dst_nodes)


In [96]:
# from scipy.stats import entropy
# import torch.nn.functional as F
# from torch import nn
# # calculate the js divergence for each row
# p = atten_mat[:, :, 50:53].numpy()
# q = atten_mat[:, :, 51:54].numpy()
# # p = p / np.sum(p, axis=1, keepdims=True)
# # q = q / np.sum(q, axis=1, keepdims=True)
# 
# # Calculate the average distribution
# m = 0.5 * (p + q)
# 
# # Calculate the Jensen-Shannon Divergence
# jsd = 0.5 * (entropy(p, m) + entropy(q, m))
# entropy(p, q)

In [97]:
# p = atten_mat
# q = atten_mat
# m = 0.5 * (p + q)
# def kl_divergence(p, q):
#     return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [8]:
import matplotlib.pyplot as plt
edge_id = 3
v = model.g.edata['v']
plt.plot(flow[edge_id, :], v[edge_id, :], '*')


In [9]:
for i in range(flow.shape[0]):
    plt.plot(flow[i, :], speed[i, :], 'o')

In [200]:
plt.plot(flow[0, :], speed[0, :], '*')
plt.plot(flow[1, :], speed[1, :], 'o')
plt.plot(flow[6, :], speed[6, :], 'o')

In [14]:
for i in range(total_flow.shape[0]):
    plt.plot(total_flow[i, :], v[i, :], 'o')

In [195]:
# distribution plot of the velocity
plt.hist(speed[1, :], bins=20)
plt.show()

In [18]:
plt.hist(speed[0, :], bins=20)
plt.show()

In [197]:
# Extract x_up and x_down from the output
with torch.no_grad():
    v_model = model.velocity_model
    up = v_model.linear12(v_model.relu(v_model.ln11(v_model.dropout(v_model.linear11(x_up)))))
    up = torch.sigmoid(up)
    
    down = v_model.linear22(v_model.relu(v_model.ln21(v_model.dropout(v_model.linear21(x_down)))))
    down = torch.sigmoid(down)

In [198]:
total_up_flow = torch.sum(x_up, dim=2).numpy()
up_flow_v = torch.sum(up, dim=2).numpy()
total_down_flow = torch.sum(x_down, dim=2).numpy()
down_flow_v = torch.sum(down, dim=2).numpy()
for i in range(total_up_flow.shape[0]):
    plt.plot(total_up_flow[i, :], up_flow_v[i, :], '*')

In [199]:
for i in range(total_up_flow.shape[0]):
    plt.plot(total_down_flow[i, :], down_flow_v[i, :], '*')

In [46]:
linear3_params = list(v_model.linear3.parameters())
# Print the parameters or do something with them
for param in linear3_params:
    print(param)

## Visualization

In [170]:
from lib.metric import masked_rmse_np, masked_mae_np, rho_risk, weighted_average_loss
x_test, y_test = sliding_win(data_dict['sc_sensor/train10'], lags=lags, horizons=pred_horizon)


In [178]:
import matplotlib.pyplot as plt
with torch.no_grad():
    # x_up = torch.FloatTensor(x_test[..., src].transpose(2, 0, 1)) # [num of src, batch_size, num_timesteps_input], num of src = num of dst = num of edges
    # x_down = torch.FloatTensor(x_test[..., dst].transpose(2, 0, 1)) # [num of dst, batch_size, num_timesteps_input]
    x = torch.FloatTensor(x_test.transpose(2, 0, 1))
    y = torch.FloatTensor(y_test.transpose(2, 0, 1))
    g.ndata['feature'] = x # [node, batch_size, num_timesteps_input]
    g.ndata['label'] = y # [node, batch_size, pred_horizon]
    # v = model.velocity_model(x_up, x_down).numpy()
    pred, multi_step_pred = model.inference(g.ndata['feature'][src], g.ndata['feature'][dst])
step = -1
node_id = 22

# label = g.ndata['label'][:,:, 0].numpy()
multi_step_pred = multi_step_pred.numpy()
label = y_test.transpose(2, 0, 1)
# label2 = y_test2.transpose(2, 0, 1)
x = np.arange(0, label.shape[1])

#adjust figure size
plt.figure(figsize=(15, 5))
plt.ylim(-5, 30)
plt.plot(x, multi_step_pred[node_id, :, step])
plt.plot(x, label[node_id, :, step])
rmse = masked_rmse_np(multi_step_pred[node_id, :, step], label[node_id, :, step])
rt = np.sum(rho_risk(multi_step_pred[node_id, :], label[node_id, :], timespan=3, rho=0.9))
# wt = weighted_average_loss(pred, label, rho=0.9, timespan=2)
plt.legend(['Prediction', 'Ground Truth'])
plt.title(f'RMSE: {rmse}')
print(f"Rho Risk: {rt}")

In [174]:
# UQ
from Diffusion_Network4_UQ import Diffusion_Model_UQ
diffusion_model_uq = Diffusion_Model_UQ(num_edges=len(src), num_timesteps_input=x_test.shape[1], graph=g, horizons=pred_horizon, scalar=None, device=device)

diffusion_model_uq.load_state_dict(torch.load('./checkpoint/diffusion/offline_diffusion_uq4_train.pth'))
if dataset_name == "crossroad":
    diffusion_model_uq.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_UQ_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
if dataset_name == "train_station":
    diffusion_model_uq.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_UQ_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
if dataset_name == "maze":
    diffusion_model_uq.load_state_dict(torch.load(f"./checkpoint/diffusion/Online_Diffusion_UQ_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))
    
with torch.no_grad():
    pred, multi_step_pred = diffusion_model_uq.inference(g.ndata['feature'][src], g.ndata['feature'][dst])
    pred, multi_step_pred = pred.numpy(), multi_step_pred.numpy()
label = y_test.transpose(2, 0, 1)
# label2 = y_test2.transpose(2, 0, 1)
x = np.arange(0, label.shape[1])

#adjust figure size
plt.figure(figsize=(15, 5))
# plt.plot(x, pred[id, :, 0])
# plt.plot(x, label[id, :, 0])
plt.plot(x, multi_step_pred[node_id, :, step])
plt.plot(x, label[node_id, :, step])
sigma = g.ndata['sigma'].numpy()
sigma = np.sqrt(pred + pred ** 2 * sigma)  # not exactly the right way to visualize the uncertainty
# plot prediction interval
# std_dev = np.sqrt(pred[node_id, :] + pred[node_id, :]**2 * sigma[node_id, :])
# plt.fill_between(x, pred[node_id, :] - 2 * sigma[node_id, :], pred[node_id, :] + 2 * std_dev, alpha=0.5)
plt.fill_between(x, pred[node_id, :] - 2 * sigma[node_id, :], pred[node_id, :] + 2 * sigma[node_id, :], alpha=0.5)
plt.ylim(-5, 40)
# plt.plot(x, v)
rmse = masked_rmse_np(multi_step_pred[node_id, :, step], label[node_id, :, step])
rt = np.sum(rho_risk(multi_step_pred[node_id, :], label[node_id, :], timespan=2, rho=0.9))
# wt = weighted_average_loss(pred.T, label.T, rho=0.9, timespan=2)
plt.title(f'RMSE: {rmse}')
print(f"Rho Risk: {rt}")

In [175]:
from scipy.stats import nbinom
ts = 270
# id = 2
# Set mean and standard deviation
mean = pred[node_id, ts]
std_dev = np.sqrt(pred[node_id, ts] + pred[node_id, ts]**2 * sigma[node_id, ts])

# Calculate shape parameter (k)
# k = (mean ** 2) / (std_dev ** 2)

# Generate x values for the plot
# x_values = np.arange(0, 2 * mean)
x_values = np.arange(0, 70)
# Calculate the probability mass function (PMF) for each x value
pmf_values = nbinom.pmf(x_values, n=sigma[node_id, ts], p=sigma[node_id, ts]/(sigma[node_id, ts] + mean))
# Plot the negative binomial distribution
plt.bar(x_values, pmf_values, label=f'Negative Binomial (mean={mean:.2f}, std_dev={std_dev:.2f})', alpha=0.7)
plt.title('Negative Binomial Distribution')
plt.xlabel('Number of People (k)')
plt.ylabel('Probability')
plt.legend()
plt.show()


In [25]:
# g.ndata['feature'] = torch.FloatTensor(x_test.transpose(2, 0, 1))
# g.ndata['label'] = torch.FloatTensor(y_test.transpose(2, 0, 1))
# z = model.transition_probability(g.ndata['feature'])
# g.ndata['embedding'] = z   # for attention
# g.apply_edges(model.transition_probability.edge_attention)

In [26]:
# with torch.no_grad():
#     num_edges = len(src)
#     sample_size = x_test.shape[0]
#     atten_mat = torch.zeros((num_nodes, num_nodes, sample_size))
#     for i in range(num_edges):
#         atten_mat[src[i], dst[i], :] = g.edata['e'][i, :]
#     atten_mat = atten_mat.numpy()

In [176]:
# LSTM
from baselines.LSTM import SimpleLSTM
lstm = SimpleLSTM(input_size=num_nodes, hidden_size=64, output_size=pred_horizon-1, num_layers=2, num_nodes=num_nodes)
lstm.load_state_dict(torch.load(f"./checkpoint/lstm/Online_LSTM_{dataset_name}_chunk{chunk_size}_lags{lags}_hor{pred_horizon}.pth"))

In [177]:
with torch.no_grad():
    multi_step_pred = lstm(torch.FloatTensor(x_test))
    multi_step_pred = multi_step_pred.reshape(multi_step_pred.shape[0], pred_horizon-1, num_nodes).permute(2, 0, 1)
    multi_step_pred = multi_step_pred.numpy()
    label = y_test.transpose(2, 0, 1)
    
    x = np.arange(0, label.shape[1])
    
    #adjust figure size
    plt.figure(figsize=(15, 5))
    plt.ylim(-5, 40)
    plt.plot(x, multi_step_pred[node_id, :, step])
    plt.plot(x, label[node_id, :, step])
    rmse = masked_rmse_np(multi_step_pred[node_id, :, step], label[node_id, :, step])
    rt = np.sum(rho_risk(multi_step_pred[node_id, :], label[node_id, :], timespan=3, rho=0.9))
    plt.legend(['Prediction', 'Ground Truth'])
    plt.title(f'RMSE: {rmse}')
print(f"Rho Risk: {rt}")

In [26]:
# xgboost
import xgboost as xgb
loaded_model = xgb.Booster()
loaded_model.load_model('./checkpoint/xgboost/offline_xgboost_train_station.model')
# loaded_model.load_model('./checkpoint/xgboost/xgboost_cross.model')

In [27]:
horizon = pred_horizon - 1
x = x_test.reshape([-1, num_input_timesteps * num_nodes])
# y = y_test.reshape([-1, pred_horizon * num_nodes])
# y_test = y_test.reshape([-1, num_input_timesteps * num_nodes])
pred = loaded_model.predict(xgb.DMatrix(x))
# pred = pred.reshape([num_nodes, -1, horizon])

pred = pred.reshape([-1, horizon, num_nodes]).transpose([2, 0, 1])
# pred = pred.T
x2 = np.arange(0, label.shape[1])
plt.figure(figsize=(15, 5))
plt.plot(x2, pred[id, :, step])  # node ts horizon
plt.plot(x2, label[id, :, step])
# plt.plot(x2, label[0, :])
rmse = masked_rmse_np(pred[id, :, step], label[id, :, step])
# rt = np.sum(rho_risk(pred[id, :], label[id, :], timespan=2, rho=0.9))
plt.title(f'RMSE: {rmse}')

In [135]:
# MA
from baselines.MA import Moving_Average
ma = Moving_Average(horizons=pred_horizon-1)
multi_step_pred = ma.inference(g.ndata['feature']).numpy()
label = y_test.transpose(2, 0, 1)
# x2 = np.arange(0, label.shape[1])
id = node_id
# step = -1
x = np.arange(0, label.shape[1])

plt.figure(figsize=(15, 5))
plt.ylim(-5, 40)
plt.plot(x, multi_step_pred[id, :, step])  # node ts horizon
plt.plot(x, label[id, :, step])
# plt.plot(x2, label[0, :])
rmse = masked_rmse_np(multi_step_pred[node_id, :, step], label[node_id, :, step])
rt = np.sum(rho_risk(multi_step_pred[node_id, :], label[node_id, :], timespan=3, rho=0.9))
plt.legend(['Prediction', 'Ground Truth'])
plt.title(f'RMSE: {rmse}')
print(f"Rho Risk: {rt}")

In [9]:
# gat
from baselines.GAT import GAT
adj_mat = g.adjacency_matrix(transpose=False, scipy_fmt="coo")
gat_model = GAT(g=adj_mat, seq_len=num_input_timesteps, feature_size=1, hidden_dim=32, out_dim=pred_horizon-1, nodes=num_nodes, num_heads=3)
gat_model.load_state_dict(torch.load("./checkpoint/gat/gat_trainstation.pth"))

In [13]:
import matplotlib.pyplot as plt
id = 2
step = 3
# up = torch.FloatTensor(x_test.transpose(2, 0, 1))[src]
# down = torch.FloatTensor(x_test.transpose(2, 0, 1))[dst]
with torch.no_grad():
    multi_step_pred = gat_model.inference(g.ndata['feature'].permute(1, 0, 2))
    multi_step_pred = multi_step_pred.numpy()
# plot prediction and ground truth

# label = g.ndata['label'][:,:, 0].numpy()
label = y_test.transpose(2, 0, 1)
# label2 = y_test2.transpose(2, 0, 1)
x = np.arange(0, label.shape[1])

#adjust figure size
plt.figure(figsize=(15, 5))
plt.plot(x, multi_step_pred[id, :, step])
plt.plot(x, label[id, :, step])

rmse = masked_rmse_np(multi_step_pred[id, :, step], label[id, :, step])
# rt = np.sum(rho_risk(pred[id, :], label[id, :], timespan=2, rho=0.9))
# wt = weighted_average_loss(pred.T, label.T, rho=0.9, timespan=2)
plt.title(f'RMSE: {rmse}')

In [43]:
# gcn
from baselines.GCN import GCN
gcn_model = GCN(in_size=num_input_timesteps, hid_size=128, out_size=pred_horizon-1, scalar=None)  # out_size: prediction horizon
gcn_model.load_state_dict(torch.load("./checkpoint/gcn/gcn_trainstation.pth"))

In [44]:
import matplotlib.pyplot as plt
id = 23
step = 3
# up = torch.FloatTensor(x_test.transpose(2, 0, 1))[src]
# down = torch.FloatTensor(x_test.transpose(2, 0, 1))[dst]
g = dgl.add_self_loop(g)
with torch.no_grad():
    multi_step_pred = gcn_model.inference(g, g.ndata['feature'])
    multi_step_pred = multi_step_pred.numpy()
# plot prediction and ground truth

# label = g.ndata['label'][:,:, 0].numpy()
label = y_test.transpose(2, 0, 1)
# label2 = y_test2.transpose(2, 0, 1)
x = np.arange(0, label.shape[1])

#adjust figure size
plt.figure(figsize=(15, 5))
plt.plot(x, multi_step_pred[id, :, step])
plt.plot(x, label[id, :, step])

rmse = masked_rmse_np(multi_step_pred[id, :, step], label[id, :, step])
# rt = np.sum(rho_risk(pred[id, :], label[id, :], timespan=2, rho=0.9))
# wt = weighted_average_loss(pred.T, label.T, rho=0.9, timespan=2)
plt.title(f'RMSE: {rmse}')

## Visualize Speed and Flow

In [223]:
x_test, y_test = sliding_win(data_dict['sc_sensor/maze14'], lags=lags, horizons=pred_horizon)
with torch.no_grad():
    x_up = torch.FloatTensor(x_test[..., src].transpose(2, 0, 1)) # [num of src, batch_size, num_timesteps_input], num of src = num of dst = num of edges
    x_down = torch.FloatTensor(x_test[..., dst].transpose(2, 0, 1)) # [num of dst, batch_size, num_timesteps_input]
    x = torch.FloatTensor(x_test.transpose(2, 0, 1))
    y = torch.FloatTensor(y_test.transpose(2, 0, 1))
    g.ndata['feature'] = x # [node, batch_size, num_timesteps_input]
    g.ndata['label'] = y # [node, batch_size, pred_horizon]
    # v = model.velocity_model(x_up, x_down).numpy()
    model(g.ndata['feature'][src], g.ndata['feature'][dst])
    # flow = torch.sum(x_up, dim=2).numpy()
    # total_flow = torch.sum(torch.cat([x_up, x_down], dim=-1), dim=-1).numpy()
flow = model.g.edata['message']
speed = model.g.edata['v']
F = model.g.edata['F']

In [226]:
# speed and F
import matplotlib.pyplot as plt
time = np.arange(speed.shape[1])
src_idx = 10
dst_idx = 6
edge_id = np.where((src == src_idx) & (dst == dst_idx))[0]
plt.plot(time, speed[edge_id, :].squeeze())
plt.plot(time, F[edge_id, :].squeeze())

In [227]:
# Flow
plt.plot(time, flow[edge_id, :].squeeze())

In [15]:
# T = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5], [0.1, 0.2, 0.3, 0.4, 0.5]])
# alpha = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
# F = 1/(1 + alpha * T)
# # f = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])
# n = torch.tensor([[1, 2, 3, 4, 5],[1, 2, 3, 4, 5]])
# for i in zip(n, F):
#     print(i)

In [16]:
# src = np.array([0, 0, 0, 3, 3, 3, 5, 5, 5, 6, 6, 6])
# dst = np.array([4, 2, 7, 1, 4, 7, 2, 7, 1, 2, 4, 1])
# g = dgl.graph((src, dst))
# g.edata['distance'] = torch.FloatTensor([43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43]) # 50m
# # save the graph
# with open('graphs/graph_data.pkl', 'wb') as file:
#     pickle.dump(g, file)

In [23]:
# 
v = 40
alpha = np.arange(-1, 1, 0.01)
F = 1/(1+alpha*v)
plt.plot(alpha, F)

In [65]:
v = 3
alpha = np.arange(0.1, 1, 0.01)
F = 1/(1+alpha*v)
plt.plot(alpha, F)